In [76]:
from pulp import *
import pandas as pd
import networkx as nx
import pandas as pd
#Heavily influenced by, need to make sure we understand the code so we can use it and not plagerize as a final product
#https://www.kaggle.com/code/itoeiji/solving-tsp-and-vrp-by-mip-using-pulp

In [57]:
distEnum = {
    "SFO": { "SFO": 0, "LAX": 500, "SJC": 75, "OAK": 50},
    "LAX": { "SFO": 500, "LAX": 0, "SJC": 280, "OAK": 800},
    "SJC": { "SFO": 75, "LAX": 280, "SJC": 0, "OAK": 30},
    "OAK": { "SFO": 50, "LAX": 800, "SJC": 30, "OAK": 0}
}


distDict = pd.DataFrame( 
    [
    [0, 500, 75, 50],
    [500, 0, 280, 800],
    [75, 280, 0, 30],
    [50, 800, 30, 0]
    ])

distDict2 = pd.DataFrame( 
    [
    [0, 9999, 469, 857, 375],  #MSY
    [9999, 0, 549, 1188, 464], #IND
    [469, 549, 0, 639, 380],   #ATL
    [857, 1188, 639, 0, 9999], #FLL
    [375, 464, 380, 9999, 0]   #MEM
    ])

In [77]:
usRoutes = pd.read_csv('../data/usRoutesClean.csv')

In [80]:
def findRoutes (dataSet, startAirport, endAirport, cutoff):
    # Create the network so that we can get our additional routes that could be used to get from our start to finish
    G = nx.from_pandas_edgelist(dataSet, 'SRCIATA', 'DESTINIATA', create_using=nx.DiGraph())

    cutoffValue = 0     # Used to track what depth the while loop is at
    noRoute = True      # Used to track if a route was found and the loop can be stopped early

    while (cutoffValue <= cutoff) & (noRoute == True):  # Loop goes through and finds the shallowest depth that we can work with
        cutoffValue = cutoffValue + 1
        paths = nx.all_simple_paths(G, startAirport, endAirport, cutoff=cutoffValue)

        for path in paths:
            noRoute = False

    if cutoffValue > cutoff:
        return("NO_PATH")

    paths = nx.all_simple_paths(G, startAirport, endAirport, cutoff=cutoffValue)

    shortestTotalDistance = 0   # Used to track what the shortest total distance is for later comparison of ranking 

    for path in paths: 
        nodeLocation = 0
        totalDistance = 0
        potentialRoute = pd.DataFrame({"SRC": [], "DESTIN": [], "METERS": []})

        for index in path:
            # Calculate total distance of the route
            currentIATA = index
            if nodeLocation != 0:
                distance = (dataSet.loc[(dataSet['SRCIATA'] == pastIATA) & (dataSet['DESTINIATA'] == currentIATA), 'meters'].values)[0]
                totalDistance = totalDistance + distance
                indexEntry = pd.DataFrame({"SRC": [pastIATA], "DESTIN": [currentIATA], "METERS": [distance]})
                potentialRoute = pd.concat([potentialRoute, indexEntry])

            nodeLocation = nodeLocation + 1
            pastIATA = index
        
        if shortestTotalDistance == 0:                  # Takes the first value found a promotes it to the shortest
            shortestTotalDistance = totalDistance
            shortestRoute = potentialRoute.copy()
        else:
            if shortestTotalDistance > totalDistance:   # Compares the shortest with the current to see if it needs to replace
                shortestTotalDistance = totalDistance
                shortestRoute = potentialRoute.copy()

    return(shortestRoute)

In [ ]:
usRoutes['SRCIATA'].sample(n=4)

3772    MSY
3559    IND
1710    ATL
4652    FLL
540     CMH
Name: SRCIATA, dtype: object

In [ ]:


test = findRoutes(usRoutes, 'MSY', 'IND', 4)
test['SRC'].iloc[1]
test['METERS']
test['DESTIN']


['BWI', 'YYC', 'LIR']

In [126]:
SRCList = []
DESTINList = []
DISTANCEList = []

sampleAirports = list(usRoutes['SRCIATA'].sample(n=3))

i = 0
while i != len(sampleAirports):
    if (i == (len(sampleAirports) - 1)):
        destAirport = sampleAirports[0]
    else:
        destAirport = sampleAirports[i+1]
    route = findRoutes(usRoutes, sampleAirports[i], destAirport, 4)

    SRCList.append(route['SRC'].iloc[0])
    DESTINList.append(route['DESTIN'].iloc[0])
    DISTANCEList.append((route['METERS'].iloc[0]/1609.344))
    if (route['DESTIN'].iloc[0] != destAirport):
        sampleAirports.insert(i+1,route['DESTIN'].iloc[0])
    i = i + 1

print(SRCList)

['DEN', 'ORD', 'AZO', 'ORD', 'PDX']


In [194]:
DistanceMatrix = []

#sampleAirports = list(usRoutes['SRCIATA'].sample(n=3))
sampleAirports = ["MSY", "IND", "ATL", "FLL"]

i = 0
while (i != len(sampleAirports)):
    DISTANCEList = []
    j = 0
    while (len(DISTANCEList) < len(sampleAirports)):
        sampleAirports = list(pd.Series(sampleAirports).unique())
    
        try:
            destAirport = sampleAirports[j+1]
        except:
            break

        if (sampleAirports[i] == sampleAirports[j]):
            DISTANCEList.append(0)
        
        route = findRoutes(usRoutes, sampleAirports[i], destAirport, 4)
        route['test'] = route['METERS']/1609.344
        #print(route)

        if (route.empty):
            DISTANCEList.append(9999)
        elif (route['DESTIN'].iloc[0] != destAirport):
            sampleAirports.insert(j+1,route['DESTIN'].iloc[0])
            DISTANCEList.append((route['METERS'].iloc[0]/1609.344))
            if(route['DESTIN'].iloc[0] == sampleAirports[j-1]):
                DISTANCEList.append(9999)
                sampleAirports.insert(j+1,route['DESTIN'].iloc[0])   
        else:
            DISTANCEList.append((route['METERS'].iloc[0]/1609.344))
        
        j = j + 1
    DistanceMatrix.append(DISTANCEList)
    i = i + 1



DistanceMatrix = pd.DataFrame(DistanceMatrix, columns = sampleAirports, index = sampleAirports)
DistanceMatrix = DistanceMatrix.fillna(0)
print(DistanceMatrix)

             MSY          ATL          MEM          IND          FLL
MSY     0.000000   374.510981   374.510981   469.207951   856.689434
ATL  9999.000000     0.000000   463.785866   379.946115   379.946115
MEM   464.050570  9999.000000     0.000000   548.701210  1186.105022
IND   379.970348   551.284250  9999.000000     0.000000   639.115689
FLL   639.440667   639.440667  1187.807579  9999.000000     0.000000


In [69]:
problem = LpProblem("LP_Test", LpMinimize)

In [70]:
x = LpVariable.dicts('x', ((i, j) for i in range(len(distDict2)) for j in range(len(distDict2))), lowBound=0, upBound=1, cat='Binary')

In [71]:
problem += pulp.lpSum(distDict2[i][j] * x[i, j] for i in range(len(distDict2)) for j in range(len(distDict2)))

In [72]:
u = pulp.LpVariable.dicts('u', (i for i in range(len(distDict2))), lowBound=1, upBound=9999, cat='Integer')

In [73]:
for i in range(len(distDict2)):
    problem += x[i, i] == 0

for i in range(len(distDict2)):
    problem += pulp.lpSum(x[i, j] for j in range(len(distDict2))) == 1
    problem += pulp.lpSum(x[j, i] for j in range(len(distDict2))) == 1

In [74]:
for i in range(len(distDict2)):
    for j in range(len(distDict2)):
        if i != j and (i != 0 and j != 0):
            problem += u[i] - u[j] <= len(distDict2) * (1 - x[i, j]) - 1

In [75]:
status = problem.solve()
status, LpStatus[status], value(problem.objective)

(1, 'Optimal', 2884.0)